In [18]:
## Imports
import pandas as pd
import numpy as np
import random

In [15]:
def create_batch():
    ## load 200 anchors images
    df_anchor = pd.read_csv("ISIC-2017_anchor_images.csv")
    #print(df_anchor.head())

    ## load 1000 (2* X, where X = 500) "unlabeled" images
    df_unlabeled = pd.read_csv("ISIC_2020_unlabeled_images.csv")
    #print(df_unlabeled.head())
    
    ## Create skeleton for the batch-file
    global df_batch
    df_batch = pd.DataFrame()
    df_temp_anchor = pd.DataFrame()
    df_temp_unlabeled_big = pd.DataFrame()
    df_temp_unlabeled_A = pd.DataFrame()
    df_temp_unlabeled_B = pd.DataFrame()

    ## Create triplets randomly in the range X (now 10)
    #
    ## First we sample X-amount of random pictures, and save their ID and their labels
    ## While also adding the url:
    # 

    #We need random anchor pictures for all 1000 rows
    #Now Anchor list is 200 and we want it to be 1000
    df_anchor = pd.concat([df_anchor]*3)
    df_temp_anchor[["image1_title", "image1_label"]] = df_anchor[["image_id", "cancer"]].sample(500)
    print("temp_anchor shuffled!")
    df_temp_anchor[["image1_url"]] = "https://github.com/vcheplygina/crowdsim/blob/main/Extra/Data/batch/" + df_temp_anchor[["image1_title"]].values + "_resized.jpg?raw=true"

    #Now unlabeled has 500 cancer and 500 non-cancer images, we want to shuffle
    df_temp_unlabeled_big = df_unlabeled[["image_id", "cancer"]].sample(frac=1)
    print("temp_unlabeled shuffled!")

    # Split into 2 groups of 500
    df_temp_unlabeled_A[["image2_title", "image2_label"]] = df_temp_unlabeled_big[0:500]
    df_temp_unlabeled_A[["image2_url"]] = "https://github.com/vcheplygina/crowdsim/blob/main/Extra/Data/batch/" + df_temp_unlabeled_A[["image2_title"]].values + "_resized.jpg?raw=true"

    df_temp_unlabeled_B[["image3_title", "image3_label"]] = df_temp_unlabeled_big[500:1000]
    df_temp_unlabeled_B[["image3_url"]] = "https://github.com/vcheplygina/crowdsim/blob/main/Extra/Data/batch/" + df_temp_unlabeled_B[["image3_title"]].values + "_resized.jpg?raw=true"

    #
    ## Then we reset their indices, and join them "horisontally"
    #

    df_temp_anchor = df_temp_anchor.reset_index(drop=True)
    df_temp_unlabeled_A = df_temp_unlabeled_A.reset_index(drop=True)
    df_temp_unlabeled_B = df_temp_unlabeled_B.reset_index(drop=True)

    #
    ## Joining here into 1 dataframe
    #
    df_batch = df_temp_anchor.join(df_temp_unlabeled_A).join(df_temp_unlabeled_B)
    print(df_batch.tail())

In [16]:
# If two "A" and "B" are the same redo:
batch_succesful = False
while not batch_succesful:
    batch_succesful = True
    create_batch()
    for i in range(100):
        if df_batch.iloc[i]['image2_title'] == df_batch.iloc[i]["image3_title"]:
            batch_succesful = False
            print ("this is bad! Trying again... count:", count)
            
print("batch created succesfully!")

temp_anchor shuffled!
temp_unlabeled shuffled!
     image1_title  image1_label  \
495  ISIC_0014603           0.0   
496  ISIC_0009993           1.0   
497  ISIC_0013578           1.0   
498  ISIC_0012735           0.0   
499  ISIC_0000463           1.0   

                                            image1_url  image2_title  \
495  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_5216586   
496  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_7186503   
497  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_1648826   
498  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_5829938   
499  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_6671773   

     image2_label                                         image2_url  \
495             1  https://github.com/vcheplygina/crowdsim/blob/m...   
496             0  https://github.com/vcheplygina/crowdsim/blob/m...   
497             0  https://github.com/vcheplygina/crowdsim/blob/m...   
498             1  ht

In [17]:
## Create the csv file:
df_batch.to_csv("batch_for_crowdsim.csv")

In [24]:
## Split batch into 10 smaller batches

df_big_batch = pd.read_csv("batch_for_crowdsim.csv")
for i in range(10):
    df_small_batch = df_big_batch[i*50:(i+1)*50]
    df_small_batch.to_csv("batch_a_0"+str(i+1)+".csv")